In [1]:
"e Juju disse: UI9IHBMKMOK  IK J  D ASDASDASDA, falou 'sai' e foi pra sala"

"e Juju disse: UI9IHBMKMOK  IK J  D ASDASDASDA, falou 'sai' e foi pra sala"

In [2]:
import sys
import os
import pandas as pd

In [3]:

project_root = os.path.abspath(os.getcwd())  # current directory = root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

Testing Modules 1 by 1:

In [4]:
from src.config import Config
config = Config("configs/config_example.json")
print("trading Pair: ", config.trading_pair)
print("Interval: ", config.interval)
print("Fast MA Period: ", config.fast_ma_period)

trading Pair:  BTCUSDT
Interval:  1m
Fast MA Period:  5


In [5]:
from src.data_fetch import DataFetcher

fetcher = DataFetcher(config.base_url, config.trading_pair, config.interval)
df = fetcher.get_latest_klines(limit=10)
print(df.head())


            open_time       open       high        low      close    volume  \
0 2024-12-11 17:21:00  100752.07  100794.55  100744.56  100791.99  20.56489   
1 2024-12-11 17:22:00  100791.99  100791.99  100603.66  100674.39  78.84379   
2 2024-12-11 17:23:00  100674.39  100727.92  100665.85  100684.54  57.16145   
3 2024-12-11 17:24:00  100684.53  100750.27  100648.27  100659.13  33.54803   
4 2024-12-11 17:25:00  100659.13  100665.43  100597.86  100627.12  18.84459   

               close_time  quote_asset_volume  trades  taker_buy_base  \
0 2024-12-11 17:21:59.999        2.072265e+06    6082        12.67234   
1 2024-12-11 17:22:59.999        7.938016e+06   11507        14.03322   
2 2024-12-11 17:23:59.999        5.755834e+06    9416        16.20122   
3 2024-12-11 17:24:59.999        3.378280e+06    9823        16.23899   
4 2024-12-11 17:25:59.999        1.896254e+06    6802         5.41000   

   taker_buy_quote ignore  
0     1.276967e+06      0  
1     1.412608e+06      0  
2 

In [6]:
from src.strategy import SimpleMA_Strategy

strategy = SimpleMA_Strategy(config.fast_ma_period, config.slow_ma_period)
signal = strategy.generate_signals(df)  # Using the df you fetched above
print("Signal:", signal)


Signal: HOLD


In [7]:
from src.ml_model import MLModel
df = pd.read_csv("data/historical_data.csv")
model = MLModel()
model.train_model(df)

Model trained and saved to models/trained_model.pkl


In [8]:
from src.backtest import Backtester
df = pd.read_csv("data/historical_data.csv")
df["open_time"] = pd.to_datetime(df["open_time"])

backtester = Backtester(
    take_profit_pct=config.take_profit_pct,
    stop_loss_pct=config.stop_loss_pct,
    position_size_usd=config.position_size_usd
)

trades_df, final_balance = backtester.run_backtest(df)
print("Number of trades:", len(trades_df))
print("Final cumulative PnL (USD):", final_balance)



Model loaded successfully.


c:\Users\ferna\Documents\rep\cry_agents\src\strategy.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ma"] = df["close"].rolling(self.bb_period).mean()
c:\Users\ferna\Documents\rep\cry_agents\src\strategy.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["std"] = df["close"].rolling(self.bb_period).std()
c:\Users\ferna\Documents\rep\cry_agents\src\strategy.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

Number of trades: 5
Final cumulative PnL (USD): 0.5926154187300807


c:\Users\ferna\Documents\rep\cry_agents\src\strategy.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rsi"] = 100 - (100 / (1 + rs))
c:\Users\ferna\miniconda3\envs\study\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\ferna\Documents\rep\cry_agents\src\strategy.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ma"] = df["close"].rolling(se

In [13]:
from joblib import parallel_backend
with parallel_backend('loky'):
    # call your parallel function here
    from src.model_evaluation import ModelEvaluation
evaluator = ModelEvaluation(output_dir="model_performance")
evaluator.run_evaluation()

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
